In [22]:
using LinearAlgebraicRepresentation
Lar = LinearAlgebraicRepresentation
using IntervalTrees
using SparseArrays
using NearestNeighbors
using DataStructures
using OrderedCollections
using BenchmarkTools

## Funzione da ottimizzare

In [23]:
function biconnected_components(EV::Lar.ChainOp)

    ps = Array{Tuple{Int, Int, Int}, 1}()
    es = Array{Tuple{Int, Int}, 1}()
    todel = Array{Int, 1}()
    visited = Array{Int, 1}()
    bicon_comps = Array{Array{Int, 1}, 1}()
    hivtx = 1

    function an_edge(point) # TODO: fix bug
        # error? : BoundsError: attempt to access 0×0 SparseMatrix ...
        edges = setdiff(EV[:, point].nzind, todel)
        if length(edges) == 0
            edges = [false]
        end
        edges[1]
    end

    function get_head(edge, tail)
        setdiff(EV[edge, :].nzind, [tail])[1]
    end

    function v_to_vi(v)
        i = findfirst(t->t[1]==v, ps)
        # seems findfirst changed from 0 to Nothing
        if typeof(i) == Nothing
            return false
        elseif i == 0
            return false
        else
            return ps[i][2]
        end
    end

    push!(ps, (1,1,1))
    push!(visited, 1)
    exit = false
    while !exit
        edge = an_edge(ps[end][1])
        if edge != false
            tail = ps[end][2]
            head = get_head(edge, ps[end][1])
            hi = v_to_vi(head)
            if hi == false
                hivtx += 1
                push!(ps, (head, hivtx, ps[end][2]))
                push!(visited, head)
            else
                if hi < ps[end][3]
                    ps[end] = (ps[end][1], ps[end][2], hi)
                end
            end
            push!(es, (edge, tail))
            push!(todel, edge)
        else
            if length(ps) == 1
                found = false
                pop!(ps)
                for i in 1:size(EV,2)
                    if !(i in visited)
                        hivtx = 1
                        push!(ps, (i, hivtx, 1))
                        push!(visited, i)
                        found = true
                        break
                    end
                end
                if !found
                    exit = true
                end

            else
                if ps[end][3] == ps[end-1][2]
                    edges = Array{Int, 1}()
                    while true
                        edge, tail = pop!(es)
                        push!(edges, edge)
                        if tail == ps[end][3]
                            if length(edges) > 1
                                push!(bicon_comps, edges)
                            end
                            break
                        end
                    end

                else
                    if ps[end-1][3] > ps[end][3]
                        ps[end-1] = (ps[end-1][1], ps[end-1][2], ps[end][3])
                    end
                end
                pop!(ps)
            end
        end
    end
    bicon_comps = sort(bicon_comps, lt=(x,y)->length(x)>length(y))
    return bicon_comps
end

biconnected_components (generic function with 1 method)

## Riscrittura codice  

Questo codice era mal scritto e mal commentato quindi abbiamo sostituito l'intera funzione con altre funzioni prese da 
https://github.com/cvdlab/LinearAlgebraicRepresentation.jl/blob/master/src/graphalgorithms.jl. 

Per rendere l'output della nuova funzione biconnectedComponent uguale a quello della funzione precedente è stato modificato opportunamente il codice correggendo anche degli errori sui cat (evidentemente usabili solo in versioni più vecchie di julia). La funzione è stata testata con la precedente per vedere se gli output coincidessero. Sarebbe opportuno testarla ulteriormente.

In [24]:
function DFV_visit( VV::Lar.Cells, out::Array, count::Int, visited::Array, parent::Array, d::Array, low::Array, stack::Array, u::Int )::Array
    visited[u] = true
    count += 1
    d[u] = count
    low[u] = d[u]
    for v in VV[u]
        if ! visited[v]
            push!(stack, [(u,v)])
            parent[v] = u
            DFV_visit( VV,out,count,visited,parent,d,low,stack, v )
            if low[v] >= d[u]
                push!(out, [outputComp(stack,u,v)])
            end
            low[u] = min( low[u], low[v] )
        else
            if ! (parent[u]==v) && (d[v] < d[u])
                push!(stack, [(u,v)])
            end
            low[u] = min( low[u], d[v] )
        end
    end
    out
end

function outputComp(stack::Array, u::Int, v::Int)::Array
    out = []
    while true
        e = pop!(stack)[1]
        push!(out,e)
        if e == (u,v) 
        	break
        end
    end
    return [out] 
end

function biconnectedComponent(model)
    W,EV = model
    V = collect(1:size(W,2))
    count = 0
    stack,out = [],[]
    visited = [false for v in V]
    parent = Union{Int, Array{Any,1}}[[] for v in V]
    d = Any[0 for v in V]
    low = Any[0 for v in V]    
    VV = Lar.verts2verts(EV)
    out = Any[]
    for u in V 
        if ! visited[u] 
            out = DFV_visit( VV,out,count,visited,parent,d,low,stack, u )
        end
    end
    out = [component for component in out if length(component) >= 1]
    EVs = [map(sort∘collect,edges) for edges in cat((out...)...,dims = 1) if length(edges)>1] 
    EVs = filter(x->!isempty(x), EVs)
    bico = map(x ->sort(collect(Set(hcat(x...)))), EVs)
    return bico
end

biconnectedComponent (generic function with 1 method)

## Dati di input

In [25]:
b=[[],[]]
EV=[[1,1]]

for i=1:60
           push!(b[1],(1.0 + i*4.0))
           push!(b[2],(1.0))
           push!(b[1],(4.0 + i*4.0))
           push!(b[2],(1.0))
           push!(b[1],(1.0 + i*4.0))
           push!(b[2],(4.0))
           push!(b[1],(4.0 + i*4.0))
           push!(b[2],(4.0))
           
           push!(EV,[1+4*(i-1),2+4*(i-1)])
           push!(EV,[1+4*(i-1),3+4*(i-1)])
           push!(EV,[2+4*(i-1),4+4*(i-1)])
           push!(EV,[3+4*(i-1),4+4*(i-1)])
end

V = permutedims(reshape(hcat(b...), (length(b[1]), length(b))))
filter!(e->e!=[1,1],EV)
cop_EV = Lar.coboundary_0(EV::Lar.Cells)


240×240 SparseMatrixCSC{Int8,Int64} with 480 stored entries:
  [1  ,   1]  =  -1
  [2  ,   1]  =  -1
  [1  ,   2]  =  1
  [3  ,   2]  =  -1
  [2  ,   3]  =  1
  [4  ,   3]  =  -1
  [3  ,   4]  =  1
  [4  ,   4]  =  1
  [5  ,   5]  =  -1
  [6  ,   5]  =  -1
  [5  ,   6]  =  1
  [7  ,   6]  =  -1
  ⋮
  [235, 234]  =  -1
  [234, 235]  =  1
  [236, 235]  =  -1
  [235, 236]  =  1
  [236, 236]  =  1
  [237, 237]  =  -1
  [238, 237]  =  -1
  [237, 238]  =  1
  [239, 238]  =  -1
  [238, 239]  =  1
  [240, 239]  =  -1
  [239, 240]  =  1
  [240, 240]  =  1

## Benchmark vecchia funzione

In [26]:
@btime biconnected_components(cop_EV)

  2.047 ms (7889 allocations: 646.53 KiB)


60-element Array{Array{Int64,1},1}:
 [2, 4, 3, 1]
 [6, 8, 7, 5]
 [10, 12, 11, 9]
 [14, 16, 15, 13]
 [18, 20, 19, 17]
 [22, 24, 23, 21]
 [26, 28, 27, 25]
 [30, 32, 31, 29]
 [34, 36, 35, 33]
 [38, 40, 39, 37]
 [42, 44, 43, 41]
 [46, 48, 47, 45]
 [50, 52, 51, 49]
 ⋮
 [194, 196, 195, 193]
 [198, 200, 199, 197]
 [202, 204, 203, 201]
 [206, 208, 207, 205]
 [210, 212, 211, 209]
 [214, 216, 215, 213]
 [218, 220, 219, 217]
 [222, 224, 223, 221]
 [226, 228, 227, 225]
 [230, 232, 231, 229]
 [234, 236, 235, 233]
 [238, 240, 239, 237]

## Benchmark nuova funzione

In [27]:
@btime biconnectedComponent((V, EV))

  976.197 μs (8381 allocations: 612.91 KiB)


60-element Array{Array{Int64,1},1}:
 [1, 2, 3, 4]
 [5, 6, 7, 8]
 [9, 10, 11, 12]
 [13, 14, 15, 16]
 [17, 18, 19, 20]
 [21, 22, 23, 24]
 [25, 26, 27, 28]
 [29, 30, 31, 32]
 [33, 34, 35, 36]
 [37, 38, 39, 40]
 [41, 42, 43, 44]
 [45, 46, 47, 48]
 [49, 50, 51, 52]
 ⋮
 [193, 194, 195, 196]
 [197, 198, 199, 200]
 [201, 202, 203, 204]
 [205, 206, 207, 208]
 [209, 210, 211, 212]
 [213, 214, 215, 216]
 [217, 218, 219, 220]
 [221, 222, 223, 224]
 [225, 226, 227, 228]
 [229, 230, 231, 232]
 [233, 234, 235, 236]
 [237, 238, 239, 240]

L'ouput della nuova funzione sono gli insiemi di vertici che compongono le componenti biconnesse ordinati. La prima non forniva questo ordinamento. Non sappiamo se questo costituisca un problema o no andando avanti nella pipeline.  
  
Essendo una ricerca su grafo non abbiamo ritenuto oppurtuno parallelizzarla.